In [1]:
from dsl.dependencies import get_property_model, register_dependencies
from dsl.input.propertymodel.property_model import get_indicator_short_name
from uuid import UUID
from dataclasses import dataclass, field
from enum import Enum
import os, getpass, datetime

register_dependencies()
pm = get_property_model()

In [2]:
class Indicator(Enum):
    """I have just mapped index to internal name here (used in properties.json)"""
    vol = 0
    mol = 1
    mass = 2
    h = 3
    a = 4
    ch = 5
    ca = 6
    u = 7
    na = 8
    
    
no_uom = UUID('1fd954cb-2c56-4174-8682-60469b2b14c1')


def ind_from_short_name(short_name: str):
    if short_name == '':
        return Indicator.na
    if short_name == 'un':
        return Indicator.u
    return Indicator[short_name]


@dataclass
class Link:
    file_prop: str
    prop_guid: UUID
    uom_guid: UUID = field(default=no_uom)
    indicator: Indicator = field(default=Indicator.na)
    ratio: str = field(default_factory=lambda: 'NoRatio')
    is_included: bool = field(default=True)
        
    def __str__(self):
        return f'''
    {{
        "FileProperty": "{self.file_prop}",
        "PropertyGuid": "{str(self.prop_guid)}",
        "UomGuid": "{str(self.uom_guid)}",
        "Indicator": {self.indicator.value},
        "RatioFormat": "{self.ratio}",
        "IsIncluded": {"true" if self.is_included else "false"}
    }}'''

In [3]:
random_prop = UUID("9b68a867-79b5-4bd7-be45-5e54e521d707")  # doesn't matter which we use for excl links, but it seems to need one
all_links = [  # starting with excl cols
    Link('SITE_TYPE', random_prop, is_included=False),
    Link('RULE_ID', random_prop, is_included=False),
    Link('REASON_EXCLUDED', random_prop, is_included=False),
    Link('SITE_UID', random_prop, is_included=False),
    Link('SAMPLE_UID', random_prop, is_included=False)
]

# iterate property model
for prop in pm.properties:
    if prop['IsInternalProperty'] == 'true' or prop['AnalysisGroup'] == 'any':
        continue
        
    prop_guid = UUID(prop['Guid'])
    headers = pm.get_property_headers_by_guid(prop_guid)
    
    try:
        for header in (h for h in headers if h.ind != 'any'):  # one for each ind
            ind = ind_from_short_name(header.ind)
            metadata = next(m for m in prop['PropertyMetadata']
                            if m['PropertyIndicator'] != 'any'
                            and Indicator[m['PropertyIndicator']] == ind)
            uom_guid = UUID(metadata['DefaultUom']) if metadata['DefaultUom'] else no_uom

            all_links.append(Link(file_prop=header.format_header(),
                                  prop_guid=prop_guid,
                                  uom_guid=uom_guid,
                                  indicator=ind,
                                  ratio=header.ratio))
    except:
        raise Exception(f'Failed to load property:\n{prop}')

In [5]:

output_path = os.path.join(os.getenv('userprofile'), 'documents', 'all_props.pigi_lnk')
now_str = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f+0000")

prefix = '''{
  "ProjectProperties": [],
  "PropertyLinks": [
'''
suffix = f'''
  ],
  "CreationAuthor": "{getpass.getuser()}",
  "CreationDate": "{now_str}",
  "LastEditAuthor": "{getpass.getuser()}",
  "LastEditDate": "{now_str}",
  "Comments": "All properties (expanded by prop, analysis, ind) with default units and ratios as of time of producing. Plus a few excl columns at the start.",
  "NumberOfTemplateProperties": {len(all_links)},
  "version": "1.1.1.0"
}}'''

with open(output_path, 'w', encoding='utf-8') as f:
    f.write(prefix)
    f.write(",".join((str(link) for link in all_links)))
    f.write(suffix)
    
print(f"File with {len(all_links)} links written to: {output_path}")

File with 23613 links written to: C:\Users\chris prosser\documents\all_props.pigi_lnk
